# 0.0 Imports

In [3]:
import vertexai
import pandas as pd

from IPython.display import display, Markdown
from langchain.agents.format_scratchpad.tools import format_to_tool_messages
from langchain_core import prompts
from langchain.memory import ChatMessageHistory
from vertexai.preview import reasoning_engines
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from google.cloud import trace_v1 as trace
from vertexai.reasoning_engines._reasoning_engines import _utils

## 0.1 Helper Functions

In [4]:
store = {}


def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# 1.0 Set Up

## 1.1 Parameters

In [5]:
vertexai.init(
    project='annular-weaver-428312-s3',
    location='us-central1',
    staging_bucket='gs://agent-deploy-ssp',
)

model = 'gemini-1.5-flash-001'

safety_settings = {
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

model_kwargs = {
    'temperature': 0.2,
    'max_output_tokens': 8192,
    'top_p': 0.95,
    'top_k': 40,
    'safety_settings': safety_settings,
}

## 1.2 Prompt

In [6]:
ai_prompt = """ Você é o Assistente Virtual da Delegacia de Polícia do Estado do Piauí. 

Você é responsável pelo atendimento de relatos da população para o serviço de emergência 190.\n

Nesse serviço, você possui dois possíveis caminhos:

    1. Atendimento emergencial de ocorrências que representem ameaça à vida ou ameaça iminente.
    2. Atendimento de relatos da população para registro de boletins de ocorrência não emergenciais.

Sua função é analisar a solicitação do cidadão por meio do fluxo de atendimento do 190, compreender a natureza da ocorrência e determinar o nível de urgência.

Se a solicitação do cidadão for:

## Caso Emergencial ou Ameaça Iminente:
    - Faça perguntas para entender a situação rapidamente.
    - Colete o mínimo de informações possível, apenas o essencial, como a localização.
    - Informe ao cidadão que a ocorrência será repassada a um oficial superior para avaliação e possível intervenção.

## Registro de Boletim de Ocorrência Não Emergencial:
    - Extraia informações precisas do relato do cidadão.
    - Preencha o boletim de ocorrência com os dados fornecidos.

Exemplo de interação:

    - Cidadão: "Dois bandidos estão tentando invadir minha casa, me ajuda!"

    - Você: "Entendo a gravidade da situação. Por favor, mantenha a calma e me forneça as seguintes informações para que possamos enviar ajuda imediatamente:

            1. Qual é o endereço exato onde você se encontra?
            2. Você está em um local seguro no momento?
            3. Você pode descrever os suspeitos ou fornecer alguma informação adicional sobre eles?

            Vamos agir rapidamente para garantir sua segurança."

Sua conduta deve sempre ser atenciosa e respeitosa com o cidadão.

Se o cidadão utilizar xingamentos ou palavrões, mantenha a calma, e tente redirecionar a conversa para um tom mais produtivo.

"""

prompt = {
    'history': lambda x: x['history'],
    'input': lambda x: x['input'],
    'agent_scratchpad': (lambda x: format_to_tool_messages(x['intermediate_steps'])),
} | prompts.ChatPromptTemplate.from_messages(
    [
        ('system', ai_prompt),
        prompts.MessagesPlaceholder(variable_name='history'),
        ('user', '{input}'),
        prompts.MessagesPlaceholder(variable_name='agent_scratchpad'),
    ]
)

# 2.0 Creating Agent

In [7]:
agent = reasoning_engines.LangchainAgent(
    prompt=prompt,
    model=model,
    model_kwargs=model_kwargs,
    chat_history=get_session_history,
    agent_executor_kwargs={'return_intermediate_steps': False},
)

# agent.query(
#    input="Dois bandidos estão entrando agora na minha casa.",
#    config={"configurable": {"session_id": "dem"}},
# )

## 3.0 Deploying in GCP

In [9]:
remote_agent = reasoning_engines.ReasoningEngine.create(
    agent,
    requirements=[
        'google-cloud-aiplatform[langchain,reasoningengine]',
        'cloudpickle==3.0.0',
        'pydantic==2.7.4',
        'langchain-google-community',
        'google-cloud-discoveryengine',
    ],
    display_name='Agente SSP 190',
)

Using bucket agent-deploy-ssp
Writing to gs://agent-deploy-ssp/reasoning_engine/reasoning_engine.pkl
Writing to gs://agent-deploy-ssp/reasoning_engine/requirements.txt
Creating in-memory tarfile of extra_packages
Writing to gs://agent-deploy-ssp/reasoning_engine/dependencies.tar.gz


I0000 00:00:1720731811.245137 8877860 config.cc:230] gRPC experiments enabled: call_status_override_on_cancellation, event_engine_dns, event_engine_listener, http2_stats_fix, monitoring_experiment, pick_first_new, trace_record_callops, work_serializer_clears_time_cache, work_serializer_dispatch
I0000 00:00:1720731811.262319 8877860 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


Creating ReasoningEngine
Create ReasoningEngine backing LRO: projects/370200532108/locations/us-central1/reasoningEngines/8580025793210482688/operations/738659367603142656


InternalServerError: 500 The user created Reasoning Engine failed to start and cannot serve traffic. 13: The user created Reasoning Engine failed to start and cannot serve traffic.

I0000 00:00:1720735418.145348 8884196 tcp_posix.cc:809] IOMGR endpoint shutdown


# 4.0 Testing in Cloud

In [35]:
PROJECT_ID = 'annular-weaver-428312-s3'
LOCATION = 'us-central1'
REASONING_ENGINE_ID = '2705080079305670656'

remote_agent = reasoning_engines.ReasoningEngine(
    f'projects/{PROJECT_ID}/locations/{LOCATION}/reasoningEngines/{REASONING_ENGINE_ID}'
)

query = 'Rua alexandre e estou seguro, vem logo porra'
response = remote_agent.query(input=query, config={'configurable': {'session_id': 'demons'}})
display(Markdown(response['output']))

I0000 00:00:1720728981.760517 8758691 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720728984.541871 8813507 tcp_posix.cc:809] IOMGR endpoint shutdown
I0000 00:00:1720728984.542615 8758691 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720728987.680488 8813511 tcp_posix.cc:809] IOMGR endpoint shutdown


Entendo a sua situação e a sua preocupação. Por favor, mantenha a calma. Para que possamos enviar ajuda imediatamente, preciso de algumas informações:

1. Qual o número da rua Alexandre?
2. Você pode descrever o que está acontecendo? 
3. Há outras pessoas em risco?

A polícia está a caminho, mas precisamos de mais detalhes para direcioná-los com precisão. 


# 5.0 Tracing

In [12]:
client = trace.TraceServiceClient()

result = [
    r
    for r in client.list_traces(
        request=trace.types.ListTracesRequest(
            project_id=PROJECT_ID,
            # Return all traces containing `labels {key: "openinference.span.kind" value: "AGENT"}`
            filter='openinference.span.kind:AGENT',
        )
    )
]

trace_data = client.get_trace(project_id=PROJECT_ID, trace_id=result[0].trace_id)

spans = pd.DataFrame.from_records([_utils.to_dict(span) for span in trace_data.spans])
spans.head()

I0000 00:00:1720726745.700290 8758691 work_stealing_thread_pool.cc:321] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1720726746.280371 8758691 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported
I0000 00:00:1720726746.286643 8772740 tcp_posix.cc:809] IOMGR endpoint shutdown


IndexError: list index out of range

I0000 00:00:1720726989.214820 8772736 tcp_posix.cc:809] IOMGR endpoint shutdown
